**Concurrency:** 

Concurrency is when two or more tasks can start, run, and complete in overlapping time periods. It doesn't necessarily mean they'll ever both be running at the same instant. For example, multitasking on a single-core machine.

Parallelism is when tasks literally run at the same time, e.g., on a multicore processor.

We're going to breakdown a simple problem - how much time does it take a program to run a method without/with concurrency.

The program will run a sleep method for x time and calculate the time it took to complete by subtracting the start time (start) from the end time (finish). The result will be 2.01 seconds.  

In [2]:
import time

start = time.perf_counter()

def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

do_something(1)
do_something(1)

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 1 second(s)...
Sleeping 1 second(s)...
Finished in 2.01 second(s)


Now let's introduce a very manual form of threading that allows for detailed tweaks, but could be more efficient as seen in the advanced method known as **Thread Pool Executor**. It'll help build how the final form is an efficient universal approach to threading whenever the detailed tweaks aren't needed.

In [4]:
import time
import threading

start = time.perf_counter()

def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

#Since our thread target is expecting an argument, we used the args keyword, which accepts arguments as a list. Careful not to insert a 
# SINGLE value.   Correct = [1.5]      INcorrect = 1.5
t1 = threading.Thread(target=do_something, args=[1.5])
t2 = threading.Thread(target=do_something, args=[1.5])

#The start() method allows the threads to run concurrently along with the rest of the script. Since we want to determine the time it takes 
# to run the script, we don't want the threads to run concurrently with the rest of the script. 
t1.start()
t2.start()

#A thread can be joined in Python by calling the Thread. join() method. This has the effect of blocking the current thread until 
# the target thread that has been joined has terminated.
t1.join()
t2.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Finished in 1.5 second(s)


We can also run threading through a loop to get multiple threads running! 

For the example below, we're creating ten threads running the same do_something sleep function that will sleep for 1.5 seconds as the sleep argument. However, since the ten threads are running as joined, we observe the program takes a total of 1.51 seconds to complete, instead of the 15 seconds it would take to complete procedurally. 

In [5]:
import time
import threading

start = time.perf_counter()

def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

#create an empty list to append threads to 
threads = []

#The underscore used in the loop eliminates the 'counter' variable and allows iterations to occur without a counter.
for _ in range(10):
    t = threading.Thread(target=do_something, args=[1.5])
    t.start()
    threads.append(t)

for thread in threads:
    thread.join()


finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...Sleeping 1.5 second(s)...

Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...Sleeping 1.5 second(s)...

Finished in 1.51 second(s)


Now we're going to look at the efficient approach, Thread Pool Executing, to threading concurrently. It also makes it easier to use multiple processes, instead of threads alone, depending on the problem we're trying to solve.

We're no longer importing threading, we're now importing **concurrent.futures**

Keep in mind this does not mean the manual approach is wrong or less. Both should be used under different circumstances.

In [7]:
import concurrent.futures
import time

start = time.perf_counter()


def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'


with concurrent.futures.ThreadPoolExecutor() as executor:
    
    #To submit the function one at a time, we use the submit() method. This method schedules a function to be executed and returns a 
    #future object. The created object now allows us to check on it after it's been scheduled 
    f1 = executor.submit(do_something, 1)
    f2 = executor.submit(do_something, 1)

    #outputs the return value of the function 
    print(f1.result())
    print(f2.result())
    
    
    # for result in results:
    #     print(result)

# threads = []

# for _ in range(10):
#     t = threading.Thread(target=do_something, args=[1.5])
#     t.start()
#     threads.append(t)

# for thread in threads:
#     thread.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 1 second(s)...Sleeping 1 second(s)...

Done Sleeping...1
Done Sleeping...1
Finished in 1.01 second(s)


We can also loop this process to output multiple submits. Instead of using the traditional loop as used for the threads, we'll be using a list comprehension.

In [10]:
import concurrent.futures
import time

start = time.perf_counter()


def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'


with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5, 4, 3, 2, 1]
    results = [executor.submit(do_something, sec) for sec in secs]

    for f in concurrent.futures.as_completed(results):
        print(f.result())
    
    
    # for result in results:
    #     print(result)

# threads = []

# for _ in range(10):
#     t = threading.Thread(target=do_something, args=[1.5])
#     t.start()
#     threads.append(t)

# for thread in threads:
#     thread.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 5 second(s)...Sleeping 4 second(s)...

Sleeping 3 second(s)...
Sleeping 2 second(s)...
Sleeping 1 second(s)...
Done Sleeping...1
Done Sleeping...2
Done Sleeping...3
Done Sleeping...4
Done Sleeping...5
Finished in 5.0 second(s)


We can also use the map function :) 

In [19]:
import concurrent.futures
import time

start = time.perf_counter()


def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'


with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5, 4, 3, 2, 1]
    
    #with the map() function, we can iterate through iterable values, such as the secs list. 
    results = executor.map(do_something, secs)
    
    list(print(result) for result in results)
    
finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 5 second(s)...
Sleeping 4 second(s)...
Sleeping 3 second(s)...Sleeping 2 second(s)...

Sleeping 1 second(s)...
Done Sleeping...5
Done Sleeping...4
Done Sleeping...3
Done Sleeping...2
Done Sleeping...1
Finished in 5.01 second(s)


Please continue the rest of the youtube video explaining this. The next section is concurrent loading of multiple photos at once from unsplash. 

Link to youtube video: https://www.youtube.com/watch?v=IEEhzQoKtQU
Link to Code: https://github.com/CoreyMSchafer/code_snippets/tree/master/Python/Threading